In [13]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")

In [2]:
memory = SqliteSaver.from_conn_string(":memory:")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream({"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config):
    print(chunk)
    print("----")

for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How's life in San Francisco?", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 88, 'total_tokens': 105}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d4a8a9f-4902-4527-9bc0-2cdd1a1d4bf6-0', usage_metadata={'input_tokens': 88, 'output_tokens': 17, 'total_tokens': 105})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7ZRQF81qzcXWs8kmjYu4nROe', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 120, 'total_tokens': 140}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-79acb197-b4d2-40f9-b333-efa5f3dde90a-0', tool_

In [4]:
search = TavilySearchResults(max_results=2)

tools = [search]

In [5]:
response = model.invoke([
    HumanMessage(content="hi im bob! and i live in sf"),
])
response.content

"Hi Bob! It's great to meet you. San Francisco is an amazing city with so much to offer. What do you love most about living there?"

In [6]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [7]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_CQhQH7IMTDWZwxAWNIB2EJSC', 'type': 'tool_call'}]


### Agent

In [9]:
agent_executor = create_react_agent(model, tools)

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', id='93ca3cdc-612a-4475-94e0-e2930a6123d3'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-b27f2f9b-210f-4e55-a35a-dcc9f604e2af-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='5f10ea39-318f-4d50-8478-7654ed80ca17'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xkEKsWNb7c5dIe2jdBAhcGpr', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-387a64c3-2795-4c27-b132-01946860977a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_xkEKsWNb7c5dIe2jdBAhcGpr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the political situation like in Ukraine like at the moment?")]}
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kiGrdkI56xYZscukZMLIfw0p', 'function': {'arguments': '{"query":"current political situation in Ukraine October 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 92, 'total_tokens': 118}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0498b91c-7b56-4669-8660-91893cc0ee19-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current political situation in Ukraine October 2023'}, 'id': 'call_kiGrdkI56xYZscukZMLIfw0p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 26, 'total_tokens': 118})]}}
---
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.aljazeera.com/news/2023/10/29/russia-ukraine-war-list-of-key-events-day-612", "content": "Here is the si

### Memory

In [16]:
memory = SqliteSaver.from_conn_string(":memory:")

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "user_1000"}}